# Data Indexing and Selection

In day 2, we looked in detail at methods and tools to access, set, and modify values in NumPy arrays.
These included indexing (e.g., ``arr[2, 1]``), slicing (e.g., ``arr[:, 1:5]``), masking (e.g., ``arr[arr > 0]``), fancy indexing (e.g., ``arr[0, [1, 5]]``), and combinations thereof (e.g., ``arr[:, [1, 5]]``).
Here we'll look at similar means of accessing and modifying values in Pandas ``Series`` and ``DataFrame`` objects.
If you have used the NumPy patterns, the corresponding patterns in Pandas will feel very familiar, though there are a few quirks to be aware of.

We'll start with the simple case of the one-dimensional ``Series`` object, and then move on to the more complicated two-dimensional ``DataFrame`` object.

## Data Selection in Series

As we saw in the previous section, a ``Series`` object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary.
If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as dictionary

Like a dictionary, the ``Series`` object provides a mapping from a collection of keys to a collection of values:

In [1]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [2]:
data['b']

0.5

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

In [3]:
'a' in data

True

In [4]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

``Series`` objects can even be modified with a dictionary-like syntax.
Just as you can extend a dictionary by assigning to a new key, you can extend a ``Series`` by assigning to a new index value:

In [6]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

This easy mutability of the objects is a convenient feature: under the hood, Pandas is making decisions about memory layout and data copying that might need to take place; the user generally does not need to worry about these issues.

### Series as one-dimensional array

A ``Series`` builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, *slices*, *masking*, and *fancy indexing*.
Examples of these are as follows:

In [7]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [8]:
# slicing by implicit integer index
data[0:2]

a    0.25
b    0.50
dtype: float64

In [9]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [10]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

Among these, slicing may be the source of the most confusion.
Notice that when slicing with an explicit index (i.e., ``data['a':'c']``), the final index is *included* in the slice, while when slicing with an implicit index (i.e., ``data[0:2]``), the final index is *excluded* from the slice.

### Indexers: loc, iloc, and ix

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [11]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [12]:
# explicit index when indexing
data[1]

'a'

In [13]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [14]:
data.loc[1]

'a'

In [15]:
data.loc[1:3]

1    a
3    b
dtype: object

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

In [16]:
data.iloc[1]

'b'

In [17]:
data.iloc[1:3]

3    b
5    c
dtype: object

## Data Selection in DataFrame

Recall that a ``DataFrame`` acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of ``Series`` structures sharing the same index.
These analogies can be helpful to keep in mind as we explore data selection within this structure.

### DataFrame as a dictionary

The first analogy we will consider is the ``DataFrame`` as a dictionary of related ``Series`` objects.
Let's return to our example of areas and populations of cities:

In [18]:
area = pd.Series({'Jakarta': 664.01, 'Surabaya': 350.54, 'Medan': 265.00,
             'Bandung': 167.67, 'Makassar': 199.26})

pop =  pd.Series({'Jakarta':  9992842,
                   'Surabaya': 2806306,
                   'Medan': 2467183,
                   'Bandung': 2341097,
                   'Makassar': 1652305})

data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
Jakarta,664.01,9992842
Surabaya,350.54,2806306
Medan,265.00,2467183
Bandung,167.67,2341097
Makassar,199.26,1652305


The individual ``Series`` that make up the columns of the ``DataFrame`` can be accessed via dictionary-style indexing of the column name:

In [19]:
data['area']

Jakarta     664.01
Surabaya    350.54
Medan       265.00
Bandung     167.67
Makassar    199.26
Name: area, dtype: float64

Equivalently, we can use attribute-style access with column names that are strings:

In [20]:
data.area

Jakarta     664.01
Surabaya    350.54
Medan       265.00
Bandung     167.67
Makassar    199.26
Name: area, dtype: float64

This attribute-style column access actually accesses the exact same object as the dictionary-style access:

In [21]:
data.area is data['area']

True

Though this is a useful shorthand, keep in mind that it does not work for all cases!
For example, if the column names are not strings, or if the column names conflict with methods of the ``DataFrame``, this attribute-style access is not possible.
For example, the ``DataFrame`` has a ``pop()`` method, so ``data.pop`` will point to this rather than the ``"pop"`` column:

In [22]:
data.pop is data['pop']

False

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use ``data['pop'] = z`` rather than ``data.pop = z``).

Like with the ``Series`` objects discussed earlier, this dictionary-style syntax can also be used to modify the object, in this case adding a new column:

In [23]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
Jakarta,664.01,9992842,15049.234198
Surabaya,350.54,2806306,8005.665545
Medan,265.00,2467183,9310.124528
Bandung,167.67,2341097,13962.527584
Makassar,199.26,1652305,8292.206163


### DataFrame as two-dimensional array

As mentioned previously, we can also view the ``DataFrame`` as an enhanced two-dimensional array.
We can examine the raw underlying data array using the ``values`` attribute:

In [24]:
data.values

array([[6.64010000e+02, 9.99284200e+06, 1.50492342e+04],
       [3.50540000e+02, 2.80630600e+06, 8.00566554e+03],
       [2.65000000e+02, 2.46718300e+06, 9.31012453e+03],
       [1.67670000e+02, 2.34109700e+06, 1.39625276e+04],
       [1.99260000e+02, 1.65230500e+06, 8.29220616e+03]])

With this picture in mind, many familiar array-like observations can be done on the ``DataFrame`` itself.
For example, we can transpose the full ``DataFrame`` to swap rows and columns:

In [25]:
data.T

,Jakarta,Surabaya,Medan,Bandung,Makassar
area,6.640100e+02,3.505400e+02,2.650000e+02,1.676700e+02,1.992600e+02
pop,9.992842e+06,2.806306e+06,2.467183e+06,2.341097e+06,1.652305e+06
density,1.504923e+04,8.005666e+03,9.310125e+03,1.396253e+04,8.292206e+03


When it comes to indexing of ``DataFrame`` objects, however, it is clear that the dictionary-style indexing of columns precludes our ability to simply treat it as a NumPy array.
In particular, passing a single index to an array accesses a row:

In [26]:
data.values[0]

array([6.64010000e+02, 9.99284200e+06, 1.50492342e+04])

and passing a single "index" to a ``DataFrame`` accesses a column:

In [27]:
data['area']

Jakarta     664.01
Surabaya    350.54
Medan       265.00
Bandung     167.67
Makassar    199.26
Name: area, dtype: float64

Thus for array-style indexing, we need another convention.
Here Pandas again uses the ``loc``, ``iloc``, and ``ix`` indexers mentioned earlier.
Using the ``iloc`` indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), but the ``DataFrame`` index and column labels are maintained in the result:

In [28]:
data.iloc[:3, :2]

,area,pop
Jakarta,664.01,9992842
Surabaya,350.54,2806306
Medan,265.00,2467183


Similarly, using the ``loc`` indexer we can index the underlying data in an array-like style but using the explicit index and column names:

In [29]:
data.loc[:'Bandung', :'pop']

,area,pop
Jakarta,664.01,9992842
Surabaya,350.54,2806306
Medan,265.00,2467183
Bandung,167.67,2341097


Any of the familiar NumPy-style data access patterns can be used within these indexers.
For example, in the ``loc`` indexer we can combine masking and fancy indexing as in the following:

In [30]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
Jakarta,9992842,15049.234198
Surabaya,2806306,8005.665545
Medan,2467183,9310.124528
Bandung,2341097,13962.527584
Makassar,1652305,8292.206163


Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [31]:
data.iloc[0, 2] = 90
data

,area,pop,density
Jakarta,664.01,9992842,90.000000
Surabaya,350.54,2806306,8005.665545
Medan,265.00,2467183,9310.124528
Bandung,167.67,2341097,13962.527584
Makassar,199.26,1652305,8292.206163


To build up your fluency in Pandas data manipulation, I suggest spending some time with a simple ``DataFrame`` and exploring the types of indexing, slicing, masking, and fancy indexing that are allowed by these various indexing approaches.

### Additional indexing conventions

There are a couple extra indexing conventions that might seem at odds with the preceding discussion, but nevertheless can be very useful in practice.
First, while *indexing* refers to columns, *slicing* refers to rows:

In [32]:
data['Surabaya':'Bandung']

,area,pop,density
Surabaya,350.54,2806306,8005.665545
Medan,265.00,2467183,9310.124528
Bandung,167.67,2341097,13962.527584


Such slices can also refer to rows by number rather than by index:

In [33]:
data[1:3]

,area,pop,density
Surabaya,350.54,2806306,8005.665545
Medan,265.00,2467183,9310.124528


Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

In [34]:
data[data.density > 10000]

,area,pop,density
Bandung,167.67,2341097,13962.527584


These two conventions are syntactically similar to those on a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.

<div style="background-color:yellow; padding: 10px"><h3><span class="fa fa-flash"></span> Quick Exercise:</h3></div>

`read_csv` returned a `DataFrame`, which is somewhat similar to a spreadsheet or database table.

* Read data from `gapminder.tsv`. Select the columns `continent`, `pop`, and `GDP` for the countries `Indonesia`, `Malaysia`, and `Singapore`.
* Find out which coutries pop < 1000000 in 1952
<hr>

In [ ]:
df = pd.read_csv('gapminder.tsv', delimiter='\t')
df.head()